In [1]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk import stem  

In [2]:
def re_rule(x):
    if re.search("[a-z0-9]+",x) != None :
        return re.search("[a-z0-9]+",x).group(0)

In [3]:
stop_words = set(stopwords.words('english'))
stop_words.update(['.',',','"',"'",'?','!',':',';','(',')','[rm]','[',']','{','}'])
wordnet_lemm = stem.WordNetLemmatizer()

def NLP_process(Sentence):
    try:
        words_result = []
        words_now = word_tokenize(Sentence)
        words_result = words_now
        words_now = [i.lower() for i in words_result if i.lower() not in stop_words]
        words_result = words_now
        words_now =  [re_rule(w) for w in words_result]
        words_result = []
        for w in words_now:
            if w != None :
                words_result.append(
                    wordnet_lemm.lemmatize(w))
        return words_result
    except:
        print(Sentence)
        return([])

In [4]:
Sentence = "▪️nwot Look cool and on trend in this striped midi bodycon dress. It's easy to style up or down- pair with black heels (or sandals for a more casual look) and you're good to go! ❤️NEW! All purchases over [rm] (excluding shipping) will get a sweet surprise gift!❤️ Stretchy lace thong style panties brand new with tags."
print(NLP_process(Sentence))

['nwot', 'look', 'cool', 'trend', 'striped', 'midi', 'bodycon', 'dress', 's', 'easy', 'style', 'down', 'pair', 'black', 'heel', 'sandal', 'casual', 'look', 're', 'good', 'go', 'new', 'purchase', 'rm', 'excluding', 'shipping', 'get', 'sweet', 'surprise', 'gift', 'stretchy', 'lace', 'thong', 'style', 'panty', 'brand', 'new', 'tag']


In [5]:
Train = pd.read_csv("train.tsv",delimiter = '\t')
Test = pd.read_csv("test.tsv",delimiter = '\t')

In [6]:
Train_Meta = pd.concat([Train.train_id,Train.category_name,Train.item_description,Train.price],axis =1)
Test_Meta = pd.concat([Test.test_id, Test.category_name ,Test.item_description],axis = 1)

In [7]:
print(Train_Meta.head(1))
print(Test_Meta.head(1))

   train_id      category_name    item_description  price
0         0  Men/Tops/T-shirts  No description yet   10.0
   test_id        category_name item_description
0        0  Women/Jewelry/Rings           Size 7


In [8]:
Train_Meta = Train_Meta.rename(columns={'train_id':'id'})
Test_Meta = Test_Meta.rename(columns = {'test_id':'id'})
Train_Meta['if_Train'] = 1
Test_Meta['if_Train'] = 0
Train_and_Test = pd.concat(
    [Train_Meta.drop(['price'],axis=1),Test_Meta],axis = 0)

In [9]:
Train_and_Test.category_name = Train_and_Test.category_name.astype('category')
Category_Dict = dict( enumerate(Train_and_Test['category_name'].cat.categories))
Train_and_Test.category_name = Train_and_Test.category_name.cat.codes
Coded_Train = Train_and_Test.loc[Train_and_Test['if_Train'] == 1]
Coded_Test = Train_and_Test.loc[Train_and_Test['if_Train'] == 0]

In [10]:
Coded_Train = Coded_Train.drop(['if_Train'],axis = 1)
Coded_Test = Coded_Test.drop(['if_Train'],axis = 1)
Coded_Train['price'] = Train_Meta.price

In [11]:
del Train
del Test
del Train_Meta
del Test_Meta
del Train_and_Test

In [12]:
#determine if train include all category data
Cate_List = []
for cate in Coded_Train.category_name :
    if cate not in Cate_List:
        Cate_List.append(cate)
print(len(Cate_List))

1288


In [154]:
No
for i in range(len(Category_Dict)):
    if i not in Cate_List:
        print(i)

144
165
171
172
182
215
219
234
239
277
286
288
353
383
386
392
411
419
692
701
795
873
975


In [13]:
Coded_Train.item_description = Coded_Train.item_description.apply(NLP_process)

nan
nan
nan
nan


In [14]:
print(Coded_Train.head(1))

   id  category_name    item_description  price
0   0            829  [description, yet]   10.0


In [157]:
#根據category分開
Cate_Dict = [ dict() for i in range(len(Category_Dict))]
Split_By_Category = [None for i in range(len(Category_Dict))]

In [158]:
for i in range(len(Category_Dict)):
    Split_By_Category[i] = Coded_Train.loc[Coded_Train['category_name'] == i]

In [159]:
def updateCateDict(DF,cate_dict_list,the_index,p_mean,p_std):
    #print(DF['price'],DF['item_description'])
    the_dict = cate_dict_list[the_index]
    if p_std == 0:
        x_std = 0
    else:
        x_std = (DF['price'] - p_mean) / p_std
        if x_std == float('nan'):
            print(DF['price'],DF['item_description'])
            print(the_index,p_mean,p_std)
    for word in DF['item_description']:        
        if the_dict.get(word) == None:
            the_dict[word] = [x_std]
        else:
            the_dict[word].append(x_std)

In [160]:
import numpy as np
for  i in range(len(Category_Dict)):
    Cate_Data =  Split_By_Category[i]
    if Split_By_Category[i].empty != True and len(Split_By_Category[i]) != 1:
        price_mean = Cate_Data.price.mean()
        price_std  = Cate_Data.price.std()
        #print(Cate_Data.head(2))
        Cate_Data.apply(updateCateDict,args=(
            Cate_Dict,i,price_mean,price_std),axis = 1)
    else:
        ;#print(i)
    

In [163]:
print(Cate_Dict[438])
print((Split_By_Category[438]))

{}
            id  category_name  \
955791  955791            438   

                                         item_description  price  
955791  [100, organic, really, work, come, bamboo, bio...   20.0  


In [164]:
for  i in range(len(Category_Dict)):
    if Cate_Dict[i] != dict():
        for key in Cate_Dict[i]:
            try:
                value = (np.mean(Cate_Dict[i][key]))
                Cate_Dict[i][key] = value
            except:
                print(key,i)

In [165]:
import json
with open("Descript_Meta.json",'w') as o_f:
    json.dump(Cate_Dict,o_f)
    

In [166]:
with open("Descript_Meta.json",'r') as i_f:
    Value_Dict = json.loads(i_f.read())


In [181]:
Category_Dictionary = {}
for key in Category_Dict:
    Category_Dictionary[Category_Dict[key]] = key

In [182]:
import json
with open("Category_Meta.json",'w') as o_f:
    json.dump(Category_Dictionary,o_f)

In [183]:
with open("Category_Meta.json",'r') as i_f:
    C_Dict = json.loads(i_f.read())

In [184]:
print(C_Dict.get('Beauty/Bath & Body/Bath'))

0
